# Customer Life Time Value

## Veri Seti Hikayesi 
Online Retail II isimli veri seti İngiltere merkezli online bir satışmağazasının 01/12/2009 -09/12/2011 tarihleri arasındaki satışlarını içeriyor.Bu şirketin ürün kataloğunda hediyelik eşyalar yer alıyor. Promosyon ürünleri olarak da düşünülebilir.Çoğu müşterisinin toptancı olduğu bilgisi de mevcut.

## Değişkenler
 **InvoiceNo**: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder <br>
 **StockCode**: Ürün kodu. Her bir ürün için eşsiz numara <br>
 **Description**: Ürün ismi <br>
 **Quantity**: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir <br>
 **InvoiceDate**: Fatura tarihi ve zamanı <br>
 **UnitPrice**: Ürün fiyatı (Sterlin cinsinden)<br>
 **CustomerID**: Eşsiz müşteri numarası<br>
 **Country**: Müşterinin yaşadığı ülke<br>

In [1]:
# Gerekli importlar
import datetime as dt
import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
# Aykırı değerlerin eşiklerinin hesaplanması
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

# Aykırı değerlerin eşik değerleriyle değiştirilmesi
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [4]:
# Veri seti okutuldu. Okutma işlemi uzun sürdüğü için kopyalandı.
df_ = pd.read_excel("/Users/aslihankalyonkat/Desktop/DSMLBC/datasets/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [5]:
# Ülke olarak UK seçiyoruz.
df = df[df["Country"] == "United Kingdom"]
df.dropna(inplace=True)
df.shape

(361878, 8)

In [6]:
# Invoice no eğer C içeriyorsa iade edilen kayıt demektir. Bu nedenle kaldırıyoruz.
# Quantity değeri 0 ise ilgili faturada satın alınan bir ürün yoktur.
# Bu nedenle Quantity değeri 0'dan büyük olanları alıyoruz.
df = df[~df["Invoice"].astype(str).str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df.shape

(354345, 8)

In [7]:
# Quantity değişkeni için aykırı değerler baskılandı.
replace_with_thresholds(df, "Quantity")

In [8]:
# Price değişkeni için aykırı değerler baskılandı.
replace_with_thresholds(df, "Price")

In [9]:
# Quantity ve Price değerleri çarpılarak ilgili fatura için toplam harcama hesaplandı ve verisetine eklendi.
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.00000,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.00000,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


In [10]:
# Analizin yapıldığı tarih son alışverişten 2 gün sonrası olarak kabul edildi.
today_date = dt.datetime(2011, 12, 11)

### Lifetime Veri Yapısı
**Monetary:** Müşterinin toplam harcaması <br>
**Recency:** Son alışverişten bugüne geçen süre <br>
**Frequency:** Müşterinin alışveriş sayısı <br>
**T**: Müşterinin ilk alışverişinden bugüne geçen süre. (Bizim için müşterinin yaşı)

In [11]:
# Lifetime Veri Yapısının Hazırlanması
# monetary, recency, T ve frequency metrikleri oluşturuldu

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df.head()

cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12747.00000,52.28571,52.85714,11,381.45545
12748.00000,53.14286,53.42857,210,153.82814
12749.00000,29.85714,30.57143,5,814.48800
12820.00000,46.14286,46.71429,4,235.58500
12822.00000,2.28571,12.57143,2,474.44000


In [12]:
# BetaGeoFitter modelini kullanarak müşterilerin 1 haftalık ve 1 aylık süreçteki işlem sayısını tahminleme
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])
cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month
Customer ID,,,,,,
12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768
12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752
12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573
12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457
12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265


In [13]:
### GammaGamma modelini kullanarak müşteriden beklenen toplam kar miktarı tahminlendi.
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],cltv_df['monetary'])
cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit
Customer ID,,,,,,,
12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768,387.82285
12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752,153.97132
12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573,844.09470
12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457,247.08095
12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265,520.82819


## 6 aylık CLTV Prediction

In [14]:
# 6 aylık customer life time value değerleri hesaplandı.
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,    # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)
cltv = cltv.reset_index()
cltv.sort_values(by='clv', ascending=False)
cltv_final = cltv_df.merge(cltv, on='Customer ID', how='left')
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv
0,12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768,387.82285,1937.04614
1,12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752,153.97132,12365.79618
2,12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573,844.09470,3446.01044
3,12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457,247.08095,631.93933
4,12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265,520.82819,1612.09665


### Sonucun Yorumlanması

Customer ID'si 589 olan müşterimiz yaşı 14 frekansı 17 olmasına rağmen clv de 2.sırada çıkmasının en önemli sebeplerinden biri monetary değeridir. Fakat; Customer ID'si 14088 olan müşterimiz 589'a göre daha fazla monetary'e sahip olmasına rağmen aşağılarda yer almaktadır. Burdan anlaşılacağı üzere clv sıralamasında yaş büyük öneme sahiptir. Ama frekans ve monetarydeğerinin düşük olması durumunda müşteri genç olsa bile clv sıralamasında yukarı taşımaya yetmeyecektir.

## Farklı zaman periyodlarından oluşan CLTV analizi

In [15]:
# 2010-2011 UK müşterileri için 1 aylık CLTV hesabı

cltv_1_month = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,    # 1 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv_1_month = cltv_1_month.reset_index()
cltv_1_month.columns = ['Customer ID', 'clv_1_month']
rfm_cltv_1_month_final = cltv_final.merge(cltv_1_month, on='Customer ID', how='left')

rfm_cltv_1_month_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,clv_1_month
0,12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768,387.82285,1937.04614,336.77883
1,12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752,153.97132,12365.79618,2148.37567
2,12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573,844.09470,3446.01044,604.07100
3,12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457,247.08095,631.93933,110.12485
4,12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265,520.82819,1612.09665,286.92234


In [16]:
# 12 aylık CLTV hesabı
cltv_12_month = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12,    # 12 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)
cltv_12_month = cltv_12_month.reset_index()
cltv_12_month.columns = ['Customer ID', 'clv_12_month']
rfm_cltv_12_month_final = cltv_final.merge(cltv_12_month, on='Customer ID', how='left')

rfm_cltv_12_month_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,clv_12_month
0,12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768,387.82285,1937.04614,3698.38114
1,12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752,153.97132,12365.79618,23623.99689
2,12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573,844.09470,3446.01044,6538.82768
3,12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457,247.08095,631.93933,1204.32622
4,12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265,520.82819,1612.09665,3029.79454


### 1 aylık ve 12 aylık CLTV hesap sonuçlarının karşılaştırılması 
6 ve 7. sıradaki müşterilerimzde bir farklılık gözlenmektedir. 1 aylık cltv de Customer ID'si 14088 olan müşteri öndeyken 12 aylık tahminde Customer ID'si 13694 olan müşterimiz öne geçmiştir. Tüm değerleri yakın fakat 1 aylık tahminde 14088'in önde olmasının sebebi monetary değeri olabilir.  Ama 12 aylık tahmine bakınca her ne kadar 14088'in monetary değeri yüksek, yaşı genç olmasına rağmen 13694'ün öne geçme sebebini frekans değerinin uzun vadede daha etkili olması şeklinde yorumlayabiliriz. Bunun dışında tabloda pek bir fark gözlenmemektedir.

## Segmentasyon ve Aksiyon Önerileri

In [17]:
# CLTV'nin Standartlaştırılması
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv
0,12747.00000,52.28571,52.85714,11,381.45545,0.20248,0.80768,387.82285,1937.04614,0.02262
1,12748.00000,53.14286,53.42857,210,153.82814,3.25295,12.97752,153.97132,12365.79618,0.14437
2,12749.00000,29.85714,30.57143,5,814.48800,0.16715,0.66573,844.09470,3446.01044,0.04023
3,12820.00000,46.14286,46.71429,4,235.58500,0.10397,0.41457,247.08095,631.93933,0.00738
4,12822.00000,2.28571,12.57143,2,474.44000,0.12914,0.51265,520.82819,1612.09665,0.01882


In [18]:
# 2010-2011 UK müşterileriiçin 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırıyoruz.

cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

cltv_final.groupby("segment").agg({"count", "mean", "sum"})

Customer ID                       recency                          T                   frequency              monetary                    expected_purc_1_week                 expected_purc_1_month                 expected_average_profit                           clv                     scaled_clv               
               mean count            sum     mean count         sum     mean count         sum      mean count   sum      mean count          sum                 mean count       sum                  mean count       sum                    mean count          sum       mean count           sum       mean count      sum
segment                                                                                                                                                                                                                                                                                                                         
D       15706.17885   643 10099073.00000 22.06710   643 14189.14286 40.50900   643 26047.28571   3.06843   643  1973 177.42499   643 114084.27131              0.07102   643  45.66438               0.28285   643 181.87271               192.22096   643 123598.07757  269.46705   643  173267.31184    0.00315   643  2.02295
C       15521.40498   642  9964742.00000 30.83645   642 19797.00000 38.16555   642 24502.28571   3.99065   642  2562 260.68927   642 167362.50814              0.11951   642  76.72527               0.47592   642 305.54248               278.06975   642 178520.78092  710.82370   642  456348.81824    0.00830   642  5.32800
B       15596.33645   642 10012848.00000 29.83890   642 19156.57143 35.11704   642 22545.14286   5.45483   642  3502 351.98686   642 225975.56147              0.16142   642 103.63101               0.64250   642 412.48377               370.49558   642 237858.16146 1271.35290   642  816208.55901    0.01484   642  9.52947
A       15390.83826   643  9896309.00000 31.46056   643 20229.14286 34.51922   643 22195.85714  11.29238   643  7261 586.30221   643 376992.32312              0.27241   643 175.16157               1.08456   643 697.37431               608.69662   643 391391.92521 3806.34897   643 2447482.38877    0.04444   643 28.57506

### A ve B Segmenti için aksiyon önerileri

   **A Segment** ->  Bu segmentdeki müşterilerimiz en önemli müşterilerdir. Bundan dolayı onlara
                     kendilerini özel hissetirecek hareketler yapmalıyız. Hediye puanlar veya
                     promosyonlar tanımlayabiliriz. Sık sık küçük indirimler sağlayarak alışveriş
                     sıklıklarını artırabiliriz.

   **B Segment** -> Bu segmentteki müşterilerimizin ortalama recency A segmentine yakın olmasına
                    rağmen frekans ve monetary değerleri daha düşüktür. Paket halinde veya toptan
                    alımlarda indirim sağlayarak hem alışveriş sıklıklarını arttırmalı hem de
                    harcayacakları para miktarını arttırmalıyız.